In [1]:
pip install confluent_kafka

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ess_streaming_data_types

Note: you may need to restart the kernel to use updated packages.


In [3]:
import datetime
import logging
from streaming_data_types import serialise_wrdn
import ipywidgets as widgets
from IPython.display import display
import pprint
import json
import functools
from uuid import uuid4

import sfi_generator_v2 as sfig2

In [4]:
config = sfig2.load_config('sfi_generator_config_v2.json')

In [5]:
output_results = widgets.Output(
    layout={"border":"1px solid gray","width":"auto"}
)

In [6]:
t_proposal_id = widgets.Text(
    value=config['defaults']['proposal_id'],
    placeholder='Proposal id',
    description='Proposal Id:',
    disabled=False
)

In [7]:
t_job_id = widgets.Text(
    value="",
    placeholder='Job id',
    description='Job Id:',
    disabled=False
)

In [8]:
b_generate_job_id = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='',
    tooltip='Generate new job id',
)

In [9]:
def generate_job_id(b):
    t_job_id.value = str(uuid4())

In [10]:
b_generate_job_id.on_click(generate_job_id)

In [11]:
t_file_name = widgets.Text(
    value=config['defaults']['file_name'],
    placeholder='File Name',
    description='File Name:',
    disabled=False
)

In [12]:
ta_message_preview = widgets.Textarea(
    value='',
    placeholder='Done Writing message in json format',
    description='',
    disabled=False,
    layout={
        'width': 'auto',
        'height': '500px'
    }
)

In [13]:
l_message_preview = widgets.Label(
    value="Message Preview"
)

In [14]:
b_prep_message = widgets.Button(
    description='Prepare message',
    disabled=False,
    button_style='',
    tooltip='Create next message to send and preview it',
    icon='eye'
)

In [15]:
def prep_message_click(
    b,
    w_output=None,
    w_proposal_id=None,
    w_job_id=None,
    w_file_name=None,
    config=None
):
    config['values'] = {
        'proposal_id': w_proposal_id.value,
        'job_id': w_job_id.value,
        'file_name' : w_file_name.value
    }
    dict_message = sfig2.prep_message(config)
    formatted_message = json.dumps(dict_message, indent=4)
    w_output.value = formatted_message
    output_results.clear_output()

In [16]:
b_prep_message.on_click(
    functools.partial(
        prep_message_click,
        w_output=ta_message_preview,
        w_proposal_id=t_proposal_id,
        w_job_id=t_job_id,
        w_file_name=t_file_name,
        config=config
    )
)

In [17]:
b_send_message = widgets.Button(
    description='Send message',
    disabled=False,
    button_style='',
    tooltip='Send message in preview',
    icon='paper-plane'
)

In [18]:
def collect_output(message):
    output_results.append_stdout(message)

In [19]:
def send_message_click(b,text_area=None,config=None):
    dict_message = json.loads(text_area.value)
    serialized_message = sfig2.serialize_message(dict_message)
    sfig2.post_message(config,serialized_message,log_message=collect_output)

In [20]:
b_send_message.on_click(
    functools.partial(
        send_message_click,
        text_area=ta_message_preview,
        config=config
    )
)

In [21]:
layout = widgets.VBox(
    [
        t_proposal_id,
        widgets.HBox([
            t_job_id,
            b_generate_job_id
        ]),
        t_file_name,
        b_prep_message,
        l_message_preview,
        ta_message_preview,
        b_send_message,
        output_results
    ],
    layout={
        'weight': 'auto',
        'height': 'auto'
    }
)

In [22]:
display(layout)